In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_json('../Data/train.json')
df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [33]:
from sklearn.cross_validation import train_test_split

X = df['ingredients'].tolist()
y = df['cuisine'].tolist()

new_X = []
for l in X:
    new_X.append(', '.join(l))
    
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=.2, random_state=20)

In [34]:
from sklearn.feature_extraction.text import *

vectorizer = TfidfVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)  

tfidf_transformer = TfidfTransformer() 

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [35]:
from sklearn.preprocessing import LabelEncoder
e = LabelEncoder()
y = e.fit_transform(df.cuisine)

In [36]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train_tfidf, y_train) 

#Test Set
X_test_counts = vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
#Prediction
y_pred = logistic.predict(X_test_tfidf)

In [37]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
acc

0.7761156505342551

In [38]:
df_test = pd.read_json('../Data/test.json')

In [41]:
X_test_values = df_test['ingredients'].tolist()

new_X = []
for l in X_test_values:
    new_X.append(', '.join(l))

In [42]:
X_train_counts = vectorizer.transform(new_X)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)

In [45]:
pred = logistic.predict(X_train_tfidf)

In [55]:
pred

array(['british', 'southern_us', 'italian', ..., 'italian', 'southern_us',
       'mexican'], dtype='<U12')

In [71]:
import csv
with open('submission.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    ids = df_test['id'].values
    new_array = np.array(list(zip(ids, pred)))
    writer.writerow(['id', 'cuisine'])
    for item in new_array:
        writer.writerow(item)